# Clustering Crypto

In [43]:
# Initial imports
import requests
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import OrdinalEncoder

### Fetching Cryptocurrency Data

In [44]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [45]:
# Create a DataFrame
response = requests.get(url)
response_json = response.json()

In [46]:
crypto_df_raw = pd.DataFrame(response_json['Data']).transpose()
crypto_df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8358 entries, 42 to BNSOLD
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Id                    8358 non-null   object
 1   Url                   8358 non-null   object
 2   ImageUrl              8355 non-null   object
 3   ContentCreatedOn      8358 non-null   object
 4   Name                  8358 non-null   object
 5   Symbol                8358 non-null   object
 6   CoinName              8358 non-null   object
 7   FullName              8358 non-null   object
 8   Description           8358 non-null   object
 9   AssetTokenStatus      8358 non-null   object
 10  Algorithm             8358 non-null   object
 11  ProofType             8358 non-null   object
 12  SortOrder             8358 non-null   object
 13  Sponsored             8358 non-null   object
 14  Taxonomy              8358 non-null   object
 15  Rating                8358 non-null   ob

### Data Preprocessing

In [47]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','MaxSupply'
crypto_df = crypto_df_raw.drop(crypto_df_raw.columns.difference([
    'CoinName', 
    'Algorithm', 
    'IsTrading', 
    'ProofType', 
    'TotalCoinsMined', 
    'MaxSupply',
]), axis='columns')

crypto_df.head()

,CoinName,Algorithm,ProofType,IsTrading,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,True,41.999952,42
300,300 token,N/A,N/A,True,300,300
365,365Coin,X11,PoW/PoS,True,0,-1
404,404Coin,Scrypt,PoW/PoS,True,0,-1
433,433 Token,N/A,N/A,False,NaN,NaN


In [48]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]
crypto_df.head()

,CoinName,Algorithm,ProofType,IsTrading,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,True,41.999952,42
300,300 token,N/A,N/A,True,300,300
365,365Coin,X11,PoW/PoS,True,0,-1
404,404Coin,Scrypt,PoW/PoS,True,0,-1
611,SixEleven,SHA-256,PoW,True,0,0


In [49]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df.loc[crypto_df['Algorithm'] != 'N/A']
crypto_df.head()

,CoinName,Algorithm,ProofType,IsTrading,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,True,41.999952,42
365,365Coin,X11,PoW/PoS,True,0,-1
404,404Coin,Scrypt,PoW/PoS,True,0,-1
611,SixEleven,SHA-256,PoW,True,0,0
808,808,SHA-256,PoW/PoS,True,0,0


In [50]:
# Remove the "IsTrading" column
crypto_df.drop(columns='IsTrading', inplace=True)

In [51]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] > 0]

In [52]:
# Drop rows where there are 'N/A' text values
crypto_df.replace('N/A', np.nan, inplace=True)
crypto_df.dropna(inplace=True)


In [53]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_names = crypto_df['CoinName'].to_frame()

In [54]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop(columns='CoinName', inplace=True)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,Scrypt,PoW/PoS,4.199995e+01,42.0
NSR,PoS,PoS,6.178572e+09,0.0
TRI,X13,PoW/PoS,1.916272e+05,0.0
CMTC,Scrypt,PoW,8.728300e+05,0.0
CHAT,Scrypt,PoW/PoS,1.000000e+09,-1.0


In [55]:
# Create dummy variables for text features
X = pd.get_dummies(crypto_df)
X.head()

,TotalCoinsMined,MaxSupply,Algorithm_Autolykos,Algorithm_BEP-2,Algorithm_BEP-20 Token,Algorithm_BLAKE256,Algorithm_BMW512 / Echo512,Algorithm_Blake2B + SHA3,Algorithm_Blake2b,Algorithm_C31,...,ProofType_PoW/PoSe,ProofType_PoW/nPoS,ProofType_ProgPoW/PoS,ProofType_Proof of Authority,ProofType_Proof-of-Work,ProofType_SPoS,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW,ProofType_dPoW/PoW
42,4.199995e+01,42.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NSR,6.178572e+09,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TRI,1.916272e+05,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CMTC,8.728300e+05,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHAT,1.000000e+09,-1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
# Standardize data
X_scaled = StandardScaler().fit_transform(X)
X_scaled.shape

(140, 83)

### Reducing Dimensions Using PCA

In [57]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)

In [58]:
# Create a DataFrame with the principal components data
pc_df = pd.DataFrame(
    data=pca.fit_transform(X_scaled), 
    columns=['PC1', 'PC2', 'PC3'], 
    index=crypto_df.index,
)

pc_df.head()

,PC1,PC2,PC3
42,0.222850,-1.326062,-1.343658
NSR,0.696263,-1.169299,-0.312068
TRI,0.654793,-1.974566,-1.626910
CMTC,-0.853523,0.447471,-0.364626
CHAT,0.222854,-1.326058,-1.343659


In [59]:
sum(pca.components_[2])

2.0801341930050308

### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [60]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pc_df)
    inertia.append(km.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

# Create the Elbow Curve using hvPlot
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [61]:
# Initialize the K-Means model
model = KMeans(n_clusters=5, random_state=0)

# Fit the model
model.fit(pc_df)

# Predict clusters
predictions = model.predict(pc_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = pd.concat(
    [coin_names, crypto_df, pc_df],
    join='inner', 
    axis='columns',
)
clustered_df['Class'] = predictions
clustered_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply,PC1,PC2,PC3,Class
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42.0,0.222850,-1.326062,-1.343658,0
NSR,NuShares,PoS,PoS,6.178572e+09,0.0,0.696263,-1.169299,-0.312068,0
TRI,Triangles Coin,X13,PoW/PoS,1.916272e+05,0.0,0.654793,-1.974566,-1.626910,0
CMTC,CometCoin,Scrypt,PoW,8.728300e+05,0.0,-0.853523,0.447471,-0.364626,2
CHAT,OpenChat,Scrypt,PoW/PoS,1.000000e+09,-1.0,0.222854,-1.326058,-1.343659,0


### Visualizing Results

#### Scatter Plot with Tradable Cryptocurrencies

In [62]:
# Scale data to create the scatter plot
scatter_data = MinMaxScaler().fit_transform(
    clustered_df[['TotalCoinsMined', 'MaxSupply']]
)
scatterplot_df = pd.DataFrame(data=scatter_data, columns=['TotalCoinsMined', 'MaxSupply'])
scatterplot_df.insert(0, 'CoinName', coin_names.values)
scatterplot_df['Class'] = clustered_df['Class'].values
scatterplot_df.head()

,CoinName,TotalCoinsMined,MaxSupply,Class
0,42 Coin,0.000000e+00,2.047619e-12,0
1,NuShares,6.240982e-06,4.761905e-14,0
2,Triangles Coin,1.935204e-10,4.761905e-14,0
3,CometCoin,8.816040e-10,4.761905e-14,2
4,OpenChat,1.010101e-06,0.000000e+00,0


In [63]:
# Plot the PCA clusters
clustered_df.hvplot.scatter(x='PC1', y='PC2', by="Class", hover_cols=['CoinName', 'Algorithm', 'MaxSupply', 'TotalCoinsMined'])

:NdOverlay   [Class]
   :Scatter   [PC1]   (PC2,CoinName,Algorithm,MaxSupply,TotalCoinsMined)

In [64]:
# Plot the scatter with x="TotalCoinsMined" and y="MaxSupply"
scatterplot_df.hvplot.scatter(x='TotalCoinsMined', y='MaxSupply', by="Class", hover_cols='CoinName')

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (MaxSupply,CoinName)

#### Table of Tradable Cryptocurrencies

In [65]:
# Table with tradable cryptos
clustered_df.sort_values(by='CoinName').hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "MaxSupply", "TotalCoinsMined", "Class"], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,MaxSupply,TotalCoinsMined,Class]

In [66]:
# Print the total number of tradable cryptocurrencies
print(f"There are {clustered_df.shape[0]} tradable cryptocurrencies.")

There are 140 tradable cryptocurrencies.
